In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils import neo4j_driver, chunk_text, chat, embed

import json
import requests

from tqdm import tqdm
from typing import List, Dict

In [ ]:
import requests

url = "https://www.gutenberg.org/cache/epub/1727/pg1727.txt"
response = requests.get(url)
if response.status_code == 200:
    with open("pg1727.txt", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("ファイルを保存しました: pg1727.txt")
else:
    print(f"ダウンロード失敗: {response.status_code}")

In [ ]:
def chunk_into_books(text: str) -> List[str]:
    """
    テキストを本ごとにチャンクする関数
    """
    return (
        text.split("PREFACE TO FIRST EDITION")[2]
        .split("FOOTNOTES")[0]
        .strip()
        .split("\nBOOK")[1:]
    )

books = chunk_into_books(response.text)

In [ ]:
chunked_books = [chunk_text(book, 1000, 40) for book in books]

In [ ]:
import ch07_tools

ENTITY_TYPES = [
    "PERSON",
    "ORGANIZATION",
    "LOCATION",
    "GOD",
    "EVENT",
    "CREATURE",
    "WEAPON_OR_TOOL"
]

def extract_entities(text: str) -> List[Dict]:
    messages = [
        {
            "role": "user",
            "content": ch07_tools.create_extraction_prompt(ENTITY_TYPES, text)
        }
    ]
    print(messages)
    output = chat(messages)
    print(f"Extraction output: {output}")
    return ch07_tools.parse_extraction_output(output)

In [ ]:
import importlib
import ch07_tools
importlib.reload(ch07_tools)

number_of_books = 1
for book_i, book in enumerate(tqdm(chunked_books[:number_of_books], desc="Processing books")):
    for chunk_i, chunk in enumerate(tqdm(book, desc=f"Book {book_i}", leave=False)):
        nodes, relationships = extract_entities(chunk)
        print(f"nodes: {nodes}, relationships: {relationships}")
        neo4j_driver.execute_query(
            ch07_tools.import_nodes_query,
            data=nodes,
            book_id=book_i,
            text=chunk,
            chunk_id=chunk_i
        )
        neo4j_driver.execute_query(
            ch07_tools.import_relationships_query, data=relationships
        )


In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (:`__Entity__`)
    RETURN 'entity' AS type, count(*) AS count
    UNION
    MATCH ()-[:RELATIONSHIP]->()
    RETURN 'relationship' AS type, count(*) AS count
    """
)
print([el.data() for el in data])

In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
    WHERE n.name = "ORESTES"
    RETURN n.description AS description
    """
)
print([el.data() for el in data])

In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[:RELATIONSHIP]-(m:__Entity__)
    WITH n,m, count(*) AS countOfRels
    ORDER BY countOfRels DESC LIMIT 1
    MATCH (n)-[r:RELATIONSHIP]-(m)
    RETURN n.name AS source, m.name AS target, countOfRels, collect(r.description) AS descriptions
"""
)
print([el.data() for el in data])

In [ ]:
candidates_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (e:__Entity__) WHERE size(e.description) > 1
    RETURN e.name AS entity_name, e.description AS description_list
    """
)
summaries = []
for candidate in tqdm(candidates_to_summarize, desc="Summarizing candidates"):
    messages = [
        {
            "role": "user",
            "content": ch07_tools.get_summarize_prompt(
                candidate["entity_name"], candidate["description_list"]
            )
        }
    ]
    summary = chat(messages)
    summaries.append({
        "entity": candidate["entity_name"],
        "summary": summary
    })

ch07_tools.import_entity_summary(neo4j_driver, summaries)


In [ ]:
print(f"Summaries: {summaries}")

In [ ]:
rels_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (s:__Entity__)-[r:RELATIONSHIP]-(t:__Entity__)
    WHERE id(s) < id(t)
    WITH s.name AS source, t.name AS target, collect(r.description) AS description_list, count(*) AS count
    WHERe count > 1
    RETURN source, target, description_list
    """
)
rel_summaries = []
for candidate in tqdm(rels_to_summarize, desc="Summarizing relationships"):
    entity_name = f"{candidate['source']} relationship to {candidate['target']}"
    messages = [
        {
            "role": "user",
            "content": ch07_tools.get_summarize_prompt(
                entity_name, candidate["description_list"]
            )
        }
    ]
    summary = chat(messages)
    rel_summaries.append({
        "source": candidate["source"],
        "target": candidate["target"],
        "summary": summary
    })

ch07_tools.import_rels_summary(neo4j_driver, rel_summaries)

In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[r:SUMMARIZED_RELATIONSHIP]-(m:__Entity__)
    WHERE n.name = 'TELEMACHUS' AND m.name = 'MINERVA'
    RETURN r.summary AS summary
    """)
print(data[0]["summary"])

In [ ]:
community_distribution = ch07_tools.calculate_communities(neo4j_driver)[0]
print(f"There are {community_distribution['communityCount']} communities with distribution: {community_distribution['communityDistribution']}")

In [ ]:
print(community_distribution)

In [ ]:
community_info, _, _ = neo4j_driver.execute_query(ch07_tools.community_info_query)
print(json.dumps(community_info, indent=2))

In [ ]:
community_info, _, _ = neo4j_driver.execute_query(ch07_tools.community_info_query)
print(f"Community info: {community_info}")

communities = []
for community in tqdm(community_info, desc="Summarizing communities"):
    messages = [
        {
            "role": "user",
            "content": ch07_tools.get_summarize_community_prompt(community["nodes"], community["rels"])
        }
    ]
    summary = chat(messages)
    print(f"Summary for community {community['communityId']}: {summary}")
    communities.append({
        "community": json.loads(ch07_tools.extract_json(summary)),
        "communityId": community["communityId"],
        "nodes": [el["id"] for el in community["nodes"]],
    })

neo4j_driver.execute_query(ch07_tools.import_community_query, data=communities)

In [ ]:
def global_retriever(query: str, rating_threshhold: float = 5)  -> str:
    community_data, _, _ = neo4j_driver.execute_query(
        """
        MATCH (c:__Community__)
        WHERE c.rating >= $rating_threshhold
        RETURN c.summary AS summary
        """,
        rating_threshhold=rating_threshhold
    )
    print(f"Got {len(community_data)} community summaries")
    intermidiate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermidaite_messages = [
            {
                "role": "system",
                "content": ch07_tools.get_map_system_prompt(community["summary"])
            },
            {
                "role": "user",
                "content": query,
            }
        ]
        intermidaite_response = chat(intermidaite_messages)
        print(f"Intermidiate response: {intermidaite_response}")
        intermidiate_results.append(intermidaite_response)

    final_messages = [
        {
            "role": "system",
            "content": ch07_tools.get_reduce_system_prompt(intermidiate_results),
        },
        {
            "role": "user",
            "content": query,
        }
    ]
    summary = chat(final_messages)    
    return summary


In [ ]:
print(global_retriever("What is this story about?"))

In [ ]:
entities, _, _ = neo4j_driver.execute_query(
    """
MATCH (e:__Entity__)
WITH 
  CASE 
    WHEN e.summary IS NOT NULL THEN e.summary 
    WHEN e.description IS NOT NULL AND size(e.description) > 0 THEN e.description[0] 
    ELSE null 
  END AS summary,
  e.name AS name
WHERE summary IS NOT NULL
RETURN summary, name

    """
)
print(entities[0]['name'])
print(json.dumps(entities, indent=2))

print({"name": entities[0]['name'], "embedding": embed(entities[0]['summary']).tolist()})

data = [{"name": el["name"], "embedding": embed(el["summary"]).tolist()} for el in entities]
neo4j_driver.execute_query(
    """UNWIND $data AS row
    MATCH (e:__Entity__ {name: row.name})
    CALL db.create.setNodeVectorProperty(e, 'embedding', row.embedding)
    """,
    data=data
)

neo4j_driver.execute_query(
    """
    CREATE VECTOR INDEX entities IF NOT EXISTS FOR (n:__Entity__) ON (n.embedding)"""
    ,
    data=data
)

In [ ]:
local_search_query = """
CALL db.index.vector.queryNodes('entities', $k, $embedding)
YIELD node, score
WITH collect(node) as nodes
WITH collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.text AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
collect {
    UNWIND nodes as n
    MATCH (n)-[r:SUMMARIZED_RELATIONSHIP]-(m) 
    WHERE m IN nodes
    RETURN r.summary AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
collect {
    UNWIND nodes as n
    RETURN n.summary AS descriptionText
} as entities
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: insideRels, 
       Entities: entities} AS text
"""

In [ ]:
k_entities = 5
topChunks = 3
topCommunities = 3
topInsideRels = 3

def local_search(query: str) -> str:
    context, _, _ = neo4j_driver.execute_query(
        local_search_query,
        embedding=embed(query).tolist(),
        topChunks=topChunks,
        topCommunities=topCommunities,
        topInsideRels=topInsideRels,
        k=k_entities,
    )
    context_str = str(context[0]["text"])
    local_messages = [
        {
            "role": "system",
            "content": ch07_tools.get_local_system_prompt(context_str)
        },
        {
            "role": "user",
            "content": query,
        }
    ]
    final_answer = chat(local_messages)
    return final_answer

In [ ]:
print(local_search("Who is Ulysses?"))